In [2]:
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
# import torch.utils.data.distributed
# import torchvision.transforms as transforms
# import torchvision.datasets as datasets
# import torchvision.models as models

from torch.utils import data
import random
import numpy as np
from itertools import product
from torch.autograd import Variable
#from dataset import Dataset

In [3]:
print(torch.__version__)

0.4.1


In [43]:
class args:
    lr = 0.1
    momentum = 0.9
    weight_decay = 1e-4
    start_epoch = 0
    epochs = 10
    gpu = None
    print_freq = 200

In [5]:
class Dataset(data.Dataset):
    def __init__(self, lists):
        'Initialization'
        self.IDs = lists

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.IDs[index]
        d_box=np.load('/scratch/xz2139/cosmo_dark/arrays/'+str(ID[0])+'_'+str(ID[1])+'_'+str(ID[2])+'.npy')
        f_box=np.load('/scratch/xz2139/cosmo_full/arrays/'+str(ID[0])+'_'+str(ID[1])+'_'+str(ID[2])+'.npy')
        return d_box,f_box

In [6]:
class Baseline(nn.Module):

    def __init__(self, in_ch, out_ch):
        
        super(Baseline, self).__init__()
        '''
        torch.nn.Conv3d: input(N,C,D,H,W)
                            output(N,C,Dout,Hout,Wout) 
        torch.nn.AvgPool3d: input(N,C,D,H,W)
                            output(N,C,Dout,Hout,Wout)     
        '''
        '''
        nn.Conv3d(in_channels, out_channels, kernel_size)
        nn.AvgPool3d()
        '''
        self.draft_model = nn.Sequential(
            nn.Conv3d(in_ch, out_ch, 3, stride=1, padding=1),
            nn.BatchNorm3d(out_ch),
#             nn.AvgPool3d(3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(in_ch, out_ch, 3, stride=1, padding=1),
            nn.BatchNorm3d(out_ch),
#             nn.AvgPool3d(3, stride=1, padding=1),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, cube):
        cube = self.draft_model(cube)
        return cube

In [8]:
train_data = [(800, 640, 224)]
val_data = [(800, 640, 224)]
test_data = [(800, 640, 224)]

In [9]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [36]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if args.gpu is not None:
            input = input.cuda(args.gpu, non_blocking=True)
        #target = target.cuda(args.gpu, non_blocking=True)

        # compute output
        input = input.unsqueeze(dim = 1)
        target = target.unsqueeze(dim = 1)
        output = model(input.float())
        
        loss = criterion(output, target.float())

        # measure accuracy and record loss
        #prec1, prec5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        #top1.update(prec1[0], input.size(0))
        #top5.update(prec5[0], input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1, top5=top5))


In [37]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(val_loader):
            if args.gpu is not None:
                input = input.cuda(args.gpu, non_blocking=True)
            target = target.cuda(args.gpu, non_blocking=True)

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec1, prec5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), input.size(0))
            top1.update(prec1[0], input.size(0))
            top5.update(prec5[0], input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                      'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                       i, len(val_loader), batch_time=batch_time, loss=losses,
                       top1=top1, top5=top5))

        print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

In [38]:
params = {'batch_size': 3,
          'shuffle': False,
          #'shuffle': True,
          'num_workers':20}
max_epochs = 100

training_set, validation_set = Dataset(train_data), Dataset(val_data)
testing_set= Dataset(test_data)
training_generator = data.DataLoader(training_set, **params)
validation_generator = data.DataLoader(validation_set, **params)
testing_generator = data.DataLoader(testing_set, **params)

for i, (dark,full) in enumerate(testing_generator):
    dark=dark
    full=full
        

In [47]:
best_prec1 = 0
dim = 1
model = Baseline(dim, dim)
criterion = nn.MSELoss().cuda() #yueqiu
optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum,
                                weight_decay=args.weight_decay)

In [48]:
for epoch in range(args.start_epoch, args.epochs):
#     if args.distributed:
#         train_sampler.set_epoch(epoch)
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(training_generator, model, criterion, optimizer, epoch)

    # evaluate on validation set
#     prec1 = validate(validation_generator, model, criterion)

#     # remember best prec@1 and save checkpoint
#     is_best = prec1 > best_prec1
#     best_prec1 = max(prec1, best_prec1)


Epoch: [0][0/1]	Time 0.122 (0.122)	Data 0.109 (0.109)	Loss 0.1144 (0.1144)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [1][0/1]	Time 0.085 (0.085)	Data 0.073 (0.073)	Loss 0.0115 (0.0115)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [2][0/1]	Time 0.083 (0.083)	Data 0.071 (0.071)	Loss 0.0048 (0.0048)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [3][0/1]	Time 0.082 (0.082)	Data 0.071 (0.071)	Loss 0.0041 (0.0041)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [4][0/1]	Time 0.082 (0.082)	Data 0.071 (0.071)	Loss 0.0040 (0.0040)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [5][0/1]	Time 0.081 (0.081)	Data 0.070 (0.070)	Loss 0.0040 (0.0040)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [6][0/1]	Time 0.081 (0.081)	Data 0.070 (0.070)	Loss 0.0040 (0.0040)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [7][0/1]	Time 0.081 (0.081)	Data 0.071 (0.071)	Loss 0.0040 (0.0040)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [8][0/1]	Time 0.081 (0.081)	Data 0.070 (0.070)	Loss 0.004